# Competi 4
### En este cuaderno vamos a practicar la técnica de embedding sobre una columna de un dataset para predecir la esperanza de vida

### Imports necesarios

In [68]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

## 1. Importamos los datos desde Kaggle

In [69]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

#!pip install kagglehub[pandas-datasets]
# Set the path to the file you'd like to load
file_path = "Life Expectancy Data.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "kumarajarshi/life-expectancy-who",
  file_path,
)

display("Primeros 5 registros:", df.head())

'Primeros 5 registros:'

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


## 2. Análisis de la calidad de los datos

#### Vamos a usar esta función para ver la información de cada columna

In [70]:
# En primer lugar vamos definir una función que nos ayudará a analizar las columnas de un dataframe
def describe_columna(df, col):
    print(f'Columna: {col}  -  Tipo de datos: {df[col].dtype}')
    print(f'Número de valores nulos: {df[col].isnull().sum()}  -  Número de valores distintos: {df[col].nunique()}')
    print('Valores más frecuentes:')
    for i, v in df[col].value_counts().iloc[:10].items() :
        print(i, ':',v, "veces")

### 2.1 Columna Country

In [71]:
describe_columna(df, "Country")

Columna: Country  -  Tipo de datos: object
Número de valores nulos: 0  -  Número de valores distintos: 193
Valores más frecuentes:
Afghanistan : 16 veces
Peru : 16 veces
Nicaragua : 16 veces
Niger : 16 veces
Nigeria : 16 veces
Norway : 16 veces
Oman : 16 veces
Pakistan : 16 veces
Panama : 16 veces
Papua New Guinea : 16 veces


#### Vemos que hay 193 paises distintos y que generalmente vamos a tener de estos paises 16 registros para cada uno, por año

### 2.2 Columna Life expectancy

In [72]:
describe_columna(df, "Life expectancy ")

Columna: Life expectancy   -  Tipo de datos: float64
Número de valores nulos: 10  -  Número de valores distintos: 362
Valores más frecuentes:
73.0 : 45 veces
75.0 : 33 veces
78.0 : 31 veces
73.6 : 28 veces
73.9 : 25 veces
81.0 : 25 veces
76.0 : 25 veces
74.5 : 24 veces
74.7 : 24 veces
73.2 : 23 veces


#### Vamos a borrar los 10 valores nulos porque es nuestra variable objetiva

### 2.3 Columna Year expectancy

In [73]:
describe_columna(df, "Year")

Columna: Year  -  Tipo de datos: int64
Número de valores nulos: 0  -  Número de valores distintos: 16
Valores más frecuentes:
2013 : 193 veces
2015 : 183 veces
2014 : 183 veces
2012 : 183 veces
2011 : 183 veces
2010 : 183 veces
2009 : 183 veces
2008 : 183 veces
2007 : 183 veces
2006 : 183 veces


#### Están todos los datos, no haremos nada

### 2.4 Columna Adult Mortality

In [74]:
describe_columna(df, "Adult Mortality")

Columna: Adult Mortality  -  Tipo de datos: float64
Número de valores nulos: 10  -  Número de valores distintos: 425
Valores más frecuentes:
12.0 : 34 veces
14.0 : 30 veces
16.0 : 29 veces
11.0 : 25 veces
138.0 : 25 veces
19.0 : 23 veces
144.0 : 22 veces
13.0 : 21 veces
17.0 : 21 veces
15.0 : 21 veces


#### Vamos a ver los casos de esos 10 valores nulos por si podriamos hacer algo con ellos

In [75]:
df[df['Adult Mortality'].isnull()].head(10)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
624,Cook Islands,2013,Developing,NaN,NaN,0,0.01,0.000000,98.0,0,82.8,0,98.0,3.58,98.0,0.1,NaN,NaN,0.1,0.1,NaN,NaN
769,Dominica,2013,Developing,NaN,NaN,0,0.01,11.419555,96.0,0,58.4,0,96.0,5.58,96.0,0.1,722.756650,NaN,2.7,2.6,0.721,12.7
1650,Marshall Islands,2013,Developing,NaN,NaN,0,0.01,871.878317,8.0,0,81.6,0,79.0,17.24,79.0,0.1,3617.752354,NaN,0.1,0.1,NaN,0.0
1715,Monaco,2013,Developing,NaN,NaN,0,0.01,0.000000,99.0,0,NaN,0,99.0,4.30,99.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN
1812,Nauru,2013,Developing,NaN,NaN,0,0.01,15.606596,87.0,0,87.3,0,87.0,4.65,87.0,0.1,136.183210,NaN,0.1,0.1,NaN,9.6
1909,Niue,2013,Developing,NaN,NaN,0,0.01,0.000000,99.0,0,77.3,0,99.0,7.20,99.0,0.1,NaN,NaN,0.1,0.1,NaN,NaN
1958,Palau,2013,Developing,NaN,NaN,0,NaN,344.690631,99.0,0,83.3,0,99.0,9.27,99.0,0.1,1932.122370,292.0,0.1,0.1,0.779,14.2
2167,Saint Kitts and Nevis,2013,Developing,NaN,NaN,0,8.54,0.000000,97.0,0,5.2,0,96.0,6.14,96.0,0.1,NaN,NaN,3.7,3.6,0.749,13.4
2216,San Marino,2013,Developing,NaN,NaN,0,0.01,0.000000,69.0,0,NaN,0,69.0,6.50,69.0,0.1,NaN,NaN,NaN,NaN,NaN,15.1
2713,Tuvalu,2013,Developing,NaN,NaN,0,0.01,78.281203,9.0,0,79.3,0,9.0,16.61,9.0,0.1,3542.135890,1819.0,0.2,0.1,NaN,0.0


#### Parece que es en el año 2013, como los años están entre 2000 y 2015, aplicaremos la media por pais en esos casos fijándonos solo en dos años atras (2011 y 2012) y dos años adelante (2014 y 2015)

### 2.5 Columna infant deaths

In [82]:
describe_columna(df, "infant deaths")

Columna: infant deaths  -  Tipo de datos: int64
Número de valores nulos: 0  -  Número de valores distintos: 209
Valores más frecuentes:
0 : 848 veces
1 : 342 veces
2 : 203 veces
3 : 175 veces
4 : 96 veces
8 : 57 veces
7 : 53 veces
9 : 48 veces
10 : 48 veces
6 : 46 veces


#### No hay nulos, no haremos nada con esta columna

## 3. Preprocesado de los datos

#### Copiamos el dataset para hacer cambios y no modificar el original

In [76]:
df_encoded = df.copy()

### 3.1 Columna Life expectancy (Borraremos los nulos)

In [77]:
df_encoded = df_encoded.dropna(subset=["Life expectancy "])
print("Nulos: ",df_encoded["Life expectancy "].isnull().sum())

Nulos:  0


### 3.2 Columna Adult Mortality (aplicaremos la media con dos años atras y adelante de 2013)

In [81]:
# Paso 1: Identificar los países con Adult Mortality nulo en 2013
paises_nulos_2013 = df_encoded[(df_encoded['Year'] == 2013) & (df_encoded['Adult Mortality'].isna())]['Country'].unique()

# Paso 2 y 3: Para cada país, calcular la media de los años 2011, 2012, 2014, 2015 y rellenar en 2013
for pais in paises_nulos_2013:
    # Filtrar los datos del país en los años 2011, 2012, 2014, 2015
    datos_pais = df_encoded[(df_encoded['Country'] == pais) & (df_encoded['Year'].isin([2011, 2012, 2014, 2015]))]
    
    # Calcular la media de Adult Mortality en esos años
    media_pais = datos_pais['Adult Mortality'].mean()
    
    # Rellenar los valores nulos en 2013 con la media calculada
    df_encoded.loc[(df_encoded['Country'] == pais) & (df_encoded['Year'] == 2013) & (df_encoded['Adult Mortality'].isna()), 'Adult Mortality'] = media_pais

# Verificamos si aún quedan valores nulos en 2013
print("Nulos: ",df_encoded[(df_encoded['Year'] == 2013) & (df_encoded['Adult Mortality'].isna())])

Nulos:  Empty DataFrame
Columns: [Country, Year, Life expectancy , Adult Mortality, infant deaths, Alcohol, percentage expenditure, Hepatitis B, Measles ,  BMI , under-five deaths , Polio, Total expenditure, Diphtheria ,  HIV/AIDS, GDP, Population,  thinness  1-19 years,  thinness 5-9 years, Income composition of resources, Schooling, Status_Developing]
Index: []


### 3.x  Columna Status (Aplicamos One-Hot Encoding)

In [79]:
# Crear el codificador
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Aplicar One-Hot Encoding
encoded_array = encoder.fit_transform(df_encoded[['Status']])

# Convertir a DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Status']))

# Concatenar con el DataFrame original
df_encoded = pd.concat([df_encoded.drop(columns=['Status']), encoded_df], axis=1)

# Mostrar el resultado
df_encoded["Status_Developing"].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: Status_Developing, dtype: float64

#### Vamos a crear solo 1 columna con el one-hot encoding, porque con solo 1 columna tendremos la informacion de Developed o Developing de cada registro

### 3.x Columna Country (Aplicamos Embeding)